In [1]:
import pandas as pd
import json
from random import randint
from time import sleep
import requests
from tqdm import tqdm

### Clan the district, cuisine, dish columns

In [2]:
df = pd.read_csv('restaurant_clean.csv', index_col=0)

In [3]:
def stripper(text):
    return text.strip()

In [4]:
df.head()

,name,review,bookmark,smile,sad,address,district,cuisine,dish
0,Deluxe Daikiya Japanese Restaurant 極尚大喜屋日本料理,597,60355,445,23,"Shop 102, 1/F, Albion Plaza, 2-6 Granville Road,",\n Tsim Sha Tsui\n,\n Japanese\n,\n Sushi/Sashimi\n
1,The Place,520,58751,260,76,"Level L, Cordis, Hong Kong at Langham Place, 5...",\n Mong Kok\n,\n International\n,\n Seafood\n
2,LAB EAT Restaurant & Bar,620,57442,475,31,"Shop 1-2, G/F, Kiu Fung Mansion, 83-89 Kimberl...",\n Tsim Sha Tsui\n,\n Western\n,\n Seafood\n
3,Market Hotpot 鮮入圍煮,320,49070,224,24,"2/F, Dundas Square, 43H Dundas Street,",\n Mong Kok\n,\n Guangdong\n,\n Hot Pot\n
4,Sky726,594,46832,442,41,"25/F, 724-726 Nathan Road,",\n Mong Kok\n,\n French\n,\n Dessert\n


In [5]:
df['district'] = df['district'].apply(lambda x: stripper(x))
df['cuisine'] = df['cuisine'].apply(lambda x: stripper(x))
df['dish'] = df['dish'].apply(lambda x: stripper(x))

In [6]:
df.head()

,name,review,bookmark,smile,sad,address,district,cuisine,dish
0,Deluxe Daikiya Japanese Restaurant 極尚大喜屋日本料理,597,60355,445,23,"Shop 102, 1/F, Albion Plaza, 2-6 Granville Road,",Tsim Sha Tsui,Japanese,Sushi/Sashimi
1,The Place,520,58751,260,76,"Level L, Cordis, Hong Kong at Langham Place, 5...",Mong Kok,International,Seafood
2,LAB EAT Restaurant & Bar,620,57442,475,31,"Shop 1-2, G/F, Kiu Fung Mansion, 83-89 Kimberl...",Tsim Sha Tsui,Western,Seafood
3,Market Hotpot 鮮入圍煮,320,49070,224,24,"2/F, Dundas Square, 43H Dundas Street,",Mong Kok,Guangdong,Hot Pot
4,Sky726,594,46832,442,41,"25/F, 724-726 Nathan Road,",Mong Kok,French,Dessert


In [7]:
df.to_csv('restaurant_clean.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27615 entries, 0 to 27614
Data columns (total 9 columns):
name        27615 non-null object
review      27615 non-null int64
bookmark    27615 non-null int64
smile       27615 non-null int64
sad         27615 non-null int64
address     27615 non-null object
district    27615 non-null object
cuisine     27615 non-null object
dish        27615 non-null object
dtypes: int64(4), object(5)
memory usage: 2.1+ MB


In [9]:
import requests

url = "https://www.als.ogcio.gov.hk/lookup"
address =  "612-618 Nathan Rd"
params = {
    "q":address
}

requests.get(url, params=params) #200 means success

<Response [200]>

In [10]:
addresses = df['address'][:10]

In [11]:
add_dict = {i:v for i, v in enumerate(list(addresses))}

In [12]:
import numpy as np

In [13]:
type(addresses.values)

numpy.ndarray

In [14]:
def ogcioparser(address):
    data = {"q":address, "n":1} #n means only send 1 
    headers ={"Accept": "application/json"}
    api_url = "https://www.als.ogcio.gov.hk/lookup"
    res = requests.post(api_url, data=data, headers=headers, timeout=(5, 14))
    add = json.loads(res.text)
    
    return add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Northing'],\
    add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Easting'],\
    add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Longitude'],\
    add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Latitude']

In [15]:
re_dict={}

In [18]:
def ogcioparser_dict(address):
    data = {"q":address, "n":1} #n means only send 1 
    headers ={"Accept": "application/json"}
    api_url = "https://www.als.ogcio.gov.hk/lookup"
    res = requests.post(api_url, data=data, headers=headers, timeout=(5, 14))
    add = json.loads(res.text)
    add_dict = {}
    add_dict['Northing'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Northing']
    add_dict['Easting'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Easting']
    add_dict['Longitude'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Longitude']
    add_dict['Latitude'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Latitude']
    return add_dict

In [28]:
addresses = df['address']

In [29]:
all_dict = {}

In [31]:
sleep_counter = 0

In [32]:
for i, a in enumerate(tqdm(addresses)):
    all_dict[i] = ogcioparser_dict(a)
    sleep_counter +=1
    if sleep_counter % 500 == 0:
        sleep(randint(5,10))

  0%|          | 42/27615 [00:31<6:00:20,  1.28it/s]

KeyboardInterrupt: 

In [26]:
df_parse = pd.DataFrame.from_dict(all_dict, orient='index')

In [27]:
df_parse

,Northing,Easting,Longitude,Latitude
0,817834,835831,114.1726,22.2994
1,819900,835365,114.1681,22.3181
2,818187,836120,114.1754,22.3026
3,819632,835676,114.1711,22.3156
4,820233,835429,114.1687,22.3211
5,816106,833758,114.1525,22.2838
6,817849,836028,114.1745,22.2995
7,817945,835916,114.1734,22.3004
8,817728,836344,114.1776,22.2984
9,818404,835608,114.1704,22.3045


In [ ]:
df_parse.to_csv('parse.csv')

In [57]:
a,b,c,d = ogcioparser(df['address'][0])
print(a,b,c,d)

817834 835831 114.1726 22.2994


In [32]:
json.loads(res.text)

{'RequestAddress': {'AddressLine': ['Shop 102, 1/F, Albion Plaza, 2-6 Granville Road,']},
 'SuggestedAddress': [{'Address': {'PremisesAddress': {'EngPremisesAddress': {'BuildingName': 'ALBION PLAZA',
      'EngStreet': {'StreetName': 'GRANVILLE ROAD',
       'BuildingNoFrom': '2',
       'BuildingNoTo': '6'},
      'EngDistrict': {'DcDistrict': 'YTM'},
      'Region': 'KLN'},
     'ChiPremisesAddress': {'Region': '九龍',
      'ChiDistrict': {'DcDistrict': 'YTM'},
      'ChiStreet': {'StreetName': '加連威老道',
       'BuildingNoFrom': '2',
       'BuildingNoTo': '6'},
      'BuildingName': '愛賓商業大廈'},
     'GeospatialInformation': [{'Northing': '817834',
       'Easting': '835831',
       'Latitude': '22.2994',
       'Longitude': '114.1726'},
      {'Northing': '817833',
       'Easting': '835833',
       'Latitude': '22.2994',
       'Longitude': '114.1726'}]}},
   'ValidationInformation': {'ValidationTime': None}}]}

In [34]:
add_1 = json.loads(res.text)

In [44]:
add_1['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Northing']

'817834'

In [45]:
add_1['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Easting']

'835831'

In [46]:
add_1['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Longitude']

'114.1726'

In [47]:
add_1['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Latitude']

'22.2994'

In [ ]:
sleep(randint(10,100))

In [49]:
len(df['address'])

27615

In [51]:
sleep_counter = 0

In [61]:
df_parse = pd.DataFrame(columns=['Longtitude','Latitude','Northing','Easting'])

In [ ]:
df_parse['Longtitude'], df_parse['Latitude'], df_parse['Northing'], df_parse['Easting'] = df['address'].apply(lambda x: ogcioparser(x))

In [58]:
for i in tqdm(range(len(df['address']))):
    df_parse['Longtitude'], df_parse['Latitude'], df_parse['Northing'], df_parse['Easting'] = df['address'].apply(lambda x: ogcioparser(x))
    sleep_counter +=1
    if sleep_counter % 500 == 0:
        sleep(randint(5,10))


  0%|          | 0/27615 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
!ipython nbconvert --to=python openrice_allrestaurant.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook openrice_allrestaurant.ipynb to python
[NbConvertApp] Writing 4311 bytes to openrice_allrestaurant.py
